# Sergio's Week 3

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

Get page information

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
Page= BeautifulSoup(source, "lxml")

Get the Table information

In [3]:
Table = Page.find('table')
headings = [th.get_text().strip() for th in Table.find("tr").find_all("th")]
headings = np.array(headings)
A=headings.shape
rows = [td.get_text().strip() for td in Table.tbody.find_all("td")]
rows =np.array(rows)
B=rows.shape

Prepare the Dataframe

In [4]:
df = pd.DataFrame(np.array(rows).reshape(int(B[0]/A[0]),A[0],), columns=headings)

Ignore cells with a borough that is Not assigned.

In [5]:
df=df[df.Borough != 'Not assigned']

if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [6]:
df.Neighbourhood[df.Neighbourhood == 'Not assigned'] = df.Borough

combined into one row with the neighborhoods separated with a comma

In [7]:
df=df.groupby(['Postcode','Borough']).Neighbourhood.unique().apply(lambda x: ', '.join(x)).reset_index()
#df[0:3]

Print Shape

In [8]:
print(df.shape)

(103, 3)


Import geocoder and create new columns

In [ ]:
import geocoder # import geocoder
df['Latitude'] = None
df['Longitude'] = None

Obtain the coordinates

In [ ]:
Codes=df.Postcode.unique().tolist()
counter=500
while counter < 500:
    for i in range(len(Codes)):
        # initialize your variable to None
        lat_lng_coords = None
        # loop until you get the coordinates
        while (counter < 500) and (lat_lng_coords is None):
            g = geocoder.google('{}, Toronto, Ontario'.format(Codes[i]))
            lat_lng_coords = g.latlng
            counter=counter+1
            
        if lat_lng_coords != None:
            df.Latitude[i] = lat_lng_coords[0]
            df.Longitude[i] = lat_lng_coords[1]
            
            df.Latitude[df.Postcode == Codes[i]] = lat_lng_coords[0]
            df.Longitude[df.Postcode == Codes[i]] = lat_lng_coords[1]

If iterations are to many obtain data from csv

In [ ]:
if counter == 500:
    df2=pd.read_csv('https://cocl.us/Geospatial_data')
    df = df.set_index('Postcode')
    df2 = df2.set_index('Postal Code')
    df.update(df2)
    df.reset_index(inplace=True)

Results

In [ ]:
df[0:3]

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887


## Replication of analysis 

In [ ]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Use just Toronto

In [ ]:
df=df[df['Borough'].str.contains("Toronto")]

Define Map center

In [ ]:
latitude = df["Latitude"].mean()
longitude = df["Longitude"].mean()
print(latitude,longitude)

43.66726218421052 -79.38988323421053


In [ ]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(df.Latitude, df.Longitude, df.Postcode):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
map_Toronto

#### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'K2FRFC31EIQNGMJQCPGSLY3KPERMRCBTN5H5SXIOPK0BJDIY' # your Foursquare ID
CLIENT_SECRET = '32XGF2LW5NBWGRDOGQLO1VKHCTW05B4QJPKOSTMNEQHJPR0Z' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Get the info for all Post Codes

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Toronto Venues

In [ ]:
Toronto_venues = getNearbyVenues(names=df['Postcode'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

#### Let's check the size of the resulting dataframe

In [ ]:
print(Toronto_venues.shape)
Toronto_venues.head()

Let's check how many venues were returned for each PostCode

In [ ]:
Toronto_venues.groupby('Postcode').count()

#### Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

##  Analyze Each Post Code

In [ ]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Postcode column back to dataframe
Toronto_onehot['Postcode'] = Toronto_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

And let's examine the new dataframe size.

In [ ]:
Toronto_onehot.shape

#### Next, let's group rows by Post Code and by taking the mean of the frequency of occurrence of each category

In [ ]:
Toronto_grouped = Toronto_onehot.groupby('Postcode').mean().reset_index()
Toronto_grouped

#### Let's print each Post Code  along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for Code in Toronto_grouped['Postcode']:
    print("----"+Code+"----")
    temp = Toronto_grouped[Toronto_grouped['Postcode'] == Code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each Post Code.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Postcode_venues_sorted = pd.DataFrame(columns=columns)
Postcode_venues_sorted['Postcode'] = Toronto_grouped['Postcode']

for ind in np.arange(Toronto_grouped.shape[0]):
    Postcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

Postcode_venues_sorted.head()

##  Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [ ]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

print(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each Post Code.

In [ ]:
# add clustering labels
Postcode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(Postcode_venues_sorted.set_index('Postcode'), on='Postcode')

Toronto_merged.head() # check the last columns!

Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Postcode'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

#### Cluster 2

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

#### Cluster 3

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

#### Cluster 4

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

#### Cluster 5

In [ ]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]